In [1]:

import os


In [2]:
import pandas as pd

In [3]:
# from datasets import load_dataset
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# # Load tokenizer & model
# model_name = "VietAI/vit5-large-vietnews-summarization"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# # Load dataset
# dataset = load_dataset("json", data_files={"train": "/kaggle/input/evidence-data/vit5_train (1).jsonl"})["train"]

# # Hàm chia văn bản thành chunk <=800 từ
# def split_into_chunks(text, chunk_size=800):
#     words = text.strip().split()
#     return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# # Tóm tắt 1 đoạn văn bản (chunk)
# def summarize(text, max_input_tokens=1024, max_output_tokens=256):
#     if not text.strip():
#         return "[EMPTY INPUT]"
    
#     # Thêm tiền tố phù hợp
#     input_text = "summarize: " + text.strip()

#     enc = tokenizer(input_text, return_tensors="pt", max_length=max_input_tokens, truncation=True).to("cuda")
#     output_ids = model.generate(
#         input_ids=enc.input_ids,
#         attention_mask=enc.attention_mask,
#         max_length=max_output_tokens,
#         num_beams=4,
#         early_stopping=True,
#         no_repeat_ngram_size=2
#     )

#     decoded = tokenizer.decode(output_ids[0], skip_special_tokens=False)
#     return decoded
# # Tóm tắt toàn bộ văn bản dài (nhiều chunk)
# def summarize_long_text(text):
#     chunks = split_into_chunks(text, chunk_size=800)
#     print(f"👉 Tổng số từ: {len(text.split())} | Số chunk: {len(chunks)}")
    
#     summaries = []
#     for i, chunk in enumerate(chunks):
#         print(f"\n🔹 Chunk {i+1} ({len(chunk.split())} từ):")
#         summary = summarize(chunk)
#         print(f"   Tóm tắt: {summary}")
#         summaries.append(summary)
    
#     # Gộp tóm tắt các chunk lại làm kết quả (có thể thêm bước tóm tắt lần 2 nếu muốn)
#     merged_summary = " ".join(summaries)
#     return merged_summary

# # # Thử trên 3 dòng đầu tiên
# # for i in range(3):
# #     input_text = dataset[i]["input"]
# #     print(f"\n==================== 📝 DÒNG {i+1} ====================")
# #     final_summary = summarize_long_text(input_text)
# #     print(f"\n✅ TÓM TẮT CUỐI CÙNG:\n{final_summary}")

# output_path = "/kaggle/working/summarized_output.jsonl"
# with open(output_path, "w", encoding="utf-8") as f_out:
#     for i, item in enumerate(dataset):
#         input_text = item["input"]
#         summary = summarize_long_text(input_text)
#         reference_output = item["output"]

#         result = {
#             "generated_summary": summary,
#             "reference_output": reference_output
#         }
        
#         f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
#         print(f"✅ Đã xử lý dòng {i+1}/{len(dataset)}")

In [4]:
import json

# Đường dẫn đến file gốc
dataset1_path = "/kaggle/input/evidence-data/vit5_train (1).jsonl"
dataset2_path = "/kaggle/input/summ-data/summarized_output.jsonl"
output_path = "/kaggle/working/summarized_output_labeled.jsonl"

# Mở cả hai file và ghi ra file mới
with open(dataset1_path, "r", encoding="utf-8") as f1, \
     open(dataset2_path, "r", encoding="utf-8") as f2, \
     open(output_path, "w", encoding="utf-8") as fout:

    for line1, line2 in zip(f1, f2):
        data1 = json.loads(line1)
        data2 = json.loads(line2)

        input_text = data1["input"]
        summary = data2["generated_summary"]

        if "Nhãn: 0.0" in input_text:
            label_text = "Đây không phải là tin đồn, hãy giải thích vì sao đây không phải tin đồn"
        elif "Nhãn: 1.0" in input_text:
            label_text = "Đây là tin đồn, hãy giải thích vì sao đây là tin đồn"
        else:
            label_text = ""

        if label_text:
            summary = label_text  + " " + summary.strip()

        new_entry = {
            "generated_summary": summary,
            "reference_output": data2["reference_output"]
        }

        fout.write(json.dumps(new_entry, ensure_ascii=False) + "\n")


In [5]:
from datasets import load_dataset

# Tải dữ liệu gốc
dataset = load_dataset("json", data_files={"data": "/kaggle/working/summarized_output_labeled.jsonl"})["data"]

# Chia train (80%) và temp (20%)
train_test = dataset.train_test_split(test_size=0.2, seed=42)

# Chia tiếp temp thành dev (10%) và test (10%)
dev_test = train_test["test"].train_test_split(test_size=0.5, seed=42)

# Tập cuối cùng
dataset_splits = {
    "train": train_test["train"],
    "validation": dev_test["train"],   # dev
    "test": dev_test["test"]
}


Generating data split: 0 examples [00:00, ? examples/s]

In [6]:
dataset_splits

{'train': Dataset({
     features: ['generated_summary', 'reference_output'],
     num_rows: 2017
 }),
 'validation': Dataset({
     features: ['generated_summary', 'reference_output'],
     num_rows: 252
 }),
 'test': Dataset({
     features: ['generated_summary', 'reference_output'],
     num_rows: 253
 })}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")

def tokenize_fn(examples):
    model_inputs = tokenizer(
        examples["generated_summary"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        examples["reference_output"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = {
    split: dataset_splits[split].map(tokenize_fn, batched=True)
    for split in ["train", "validation", "test"]
}


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/2017 [00:00<?, ? examples/s]

Map:   0%|          | 0/252 [00:00<?, ? examples/s]

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

2025-07-08 14:21:17.769542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751984478.145256      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751984478.250878      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [9]:
from datasets import Dataset

tokenized_train = tokenized_dataset["train"]
tokenized_val = tokenized_dataset["validation"]
tokenized_test = tokenized_dataset["test"]

In [10]:
from transformers import TrainingArguments, Trainer
import torch

from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./vit5_explanation_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,                  # Giới hạn số model lưu lại
    fp16=torch.cuda.is_available(),
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipykernel_19/186443093.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,9.973500
20,1.106200
30,0.755700
40,0.662800
50,0.582300
60,0.598700
70,0.597200
80,0.518700
90,0.574800
100,0.534000


TrainOutput(global_step=378, training_loss=0.7327861949880287, metrics={'train_runtime': 824.2401, 'train_samples_per_second': 7.341, 'train_steps_per_second': 0.459, 'total_flos': 3664708585390080.0, 'train_loss': 0.7327861949880287, 'epoch': 2.982178217821782})

In [11]:
trainer.save_model("./vit5_explanation_model")
tokenizer.save_pretrained("./vit5_explanation_model")


('./vit5_explanation_model/tokenizer_config.json',
 './vit5_explanation_model/special_tokens_map.json',
 './vit5_explanation_model/spiece.model',
 './vit5_explanation_model/added_tokens.json',
 './vit5_explanation_model/tokenizer.json')

In [12]:
# predictions = trainer.predict(tokenized_dataset["test"])
# preds = predictions.predictions
# labels = predictions.label_ids

# # Decode
# decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
# decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


In [13]:
# from datasets import load_metric

# rouge = load_metric("rouge")

# rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
# for key in rouge_output:
#     print(f"{key}: {rouge_output[key].mid.fmeasure:.4f}")


In [14]:
# from datasets import load_metric

# bleu = load_metric("bleu")

# # BLEU expects list of list of tokens
# bleu_input = {
#     "predictions": [pred.split() for pred in decoded_preds],
#     "references": [[label.split()] for label in decoded_labels]
# }

# bleu_output = bleu.compute(**bleu_input)
# print(f"BLEU score: {bleu_output['bleu']:.4f}")


In [15]:
# !pip install bart_score

# from bart_score import BARTScorer

# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu')
# bart_scorer.load_model()

# # Tính điểm trung bình
# scores = bart_scorer.score(decoded_preds, decoded_labels, batch_size=4)
# print(f"BARTScore (avg): {sum(scores)/len(scores):.4f}")
